In [1]:
!git clone https://github.com/yalibina/FashionMeter.git

Cloning into 'FashionMeter'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 91 (delta 26), reused 51 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 267.18 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [2]:
!cd FashionMeter && pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [8]:
!cd FashionMeter/src/dataload &&  ./download.sh

Downloading...
From (original): https://drive.google.com/uc?id=14gShE_QBI8bd6R0r1tlvEPbu2y2vMbI_
From (redirected): https://drive.google.com/uc?id=14gShE_QBI8bd6R0r1tlvEPbu2y2vMbI_&confirm=t&uuid=abc876e5-19c7-41ea-913a-41d513eea7cb
To: /content/FashionMeter/src/dataload/mmls_data.zip
100% 501M/501M [00:05<00:00, 88.2MB/s]
Archive:  mmls_data.zip
   creating: ../data/downloaded_images/
   creating: ../data/downloaded_images/comfy/
   creating: ../data/downloaded_images/goth/
   creating: ../data/downloaded_images/formal/
   creating: ../data/downloaded_images/boho/
   creating: ../data/downloaded_images/vintage/
   creating: ../data/downloaded_images/casual/
   creating: ../data/downloaded_images/military/
   creating: ../data/downloaded_images/preppy/
   creating: ../data/downloaded_images/streetwear/
   creating: ../data/downloaded_images/grunge/
   creating: ../data/downloaded_images/sporty/
  inflating: ../data/downloaded_images/comfy/a6ba17ca9930fc2f5be19138794c24f2.jpg  
  inflat

In [11]:
!pwd
!ls

/content/FashionMeter
config.yaml  examples  README.md  requirements.txt  src  training.sh


In [12]:
import yaml
import os
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
from src.dataload.dataset import (
    train_dataloader,
    val_dataloader,
    class_weights,
    ids2label,
    label2ids,
    NUM_LABELS
)
from src.models.vit import LitViT


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [13]:
pl.seed_everything(42, workers=True)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [14]:
%cd FashionMeter

[Errno 2] No such file or directory: 'FashionMeter'
/content/FashionMeter


In [15]:
!ls

config.yaml  examples  README.md  requirements.txt  src  training.sh


In [18]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [19]:
N_EPOCHS = config['N_EPOCHS']
LR = config['LR']
WD = config['WD']
CHECKPOINT_DIR = config['CHECKPOINT_DIR']
PROJECT_NAME = config['PROJECT_NAME']
MODEL_NAME = config['MODEL_NAME']

print(config)

{'N_EPOCHS': 10, 'LR': 1e-05, 'WD': 0.3, 'BATCH_SIZE': 64, 'NUM_WORKERS': 2, 'DATA_ROOT': 'src/data/downloaded_images', 'CHECKPOINT_DIR': '../checkpoints/', 'PROJECT_NAME': 'fashion_meter', 'MODEL_NAME': 'google/vit-base-patch16-224-in21k', 'TRAIN_VAL_SPLIT': 0.8}


In [20]:
wandb.init(entity='mmls05', name=f'{N_EPOCHS}epochs_lr{LR}_wd{WD}',)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yalibina (yalibina-hse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
wandb_logger = WandbLogger(
    project=PROJECT_NAME,
    checkpoint_name=f'{N_EPOCHS}epochs_lr{LR}_wd{WD}',
    log_model=True
)

trainer = pl.Trainer(
    logger=wandb_logger,
    log_every_n_steps=10,
    max_epochs=N_EPOCHS,
    deterministic=True,
    default_root_dir=CHECKPOINT_DIR,
    precision="16-mixed",
)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/plugins/precision/amp.py:55: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [22]:
model = LitViT(
    num_labels=NUM_LABELS,
    id2label=ids2label,
    label2id=label2ids,
    class_weights=class_weights,
    lr=LR,
    weight_decay=WD,
    model_name=MODEL_NAME
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type                      | Params
---------------------------------------------------
0 | vit  | ViTForImageClassification | 85.8 M
---------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params
343.228   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train/accuracy,▁▂▂▄▄▃▄▅▅▅▆▆▇▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇████▇█
train/f1,▁▂▂▄▄▃▄▄▅▅▇▆▇▆▆▆▆▇▇▇█▇▆▇▇█▇█▇▆█▇▇███▇█
train/fbeta,▁▂▂▄▄▃▄▅▆▅▇▆█▇▇▆▇▇▇▇█▇▆▇██▇██▆█▇▇███▇█
train/loss,███▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▃▄▃▃▂▃▃▂▃▂▂▁▁▂▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val/accuracy_epoch,▁▃▅▆▆▇▇▇██
val/f1_epoch,▁▃▄▆▆▇▇▇██
val/fbeta_epoch,▁▃▅▆▆▇▇▇██
val/loss_epoch,█▇▆▆▅▄▃▂▂▁
epoch,9
